# Open API

API is used when one requests data from a company, organization, etc.
* usually requires authorization from the provider
* must follow guidelines provided from the source

## Retrieving Data using API

In [1]:
import requests
import pandas as pd

In [2]:
apikey = '6f676f414f6c73683132304f4d474f6c'

In [9]:
# getting first 5 rows
url = f'http://openapi.seoul.go.kr:8088/{apikey}/json/bikeList/1/5/'
json = requests.get(url).json()
json.keys() # returns dict key of ['rentBikeStatus']

pd.DataFrame(json['rentBikeStatus']['row'])

,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId
0,7,101. (구)합정동 주민센터,4,14,37.54956055,126.90575409,ST-3
1,22,102. 망원역 1번출구 앞,7,0,37.55564880,126.91062927,ST-4
2,16,103. 망원역 2번출구 앞,3,0,37.55495071,126.91083527,ST-5
3,15,104. 합정역 1번출구 앞,0,0,37.55062866,126.91498566,ST-6
4,7,105. 합정역 5번출구 앞,1,0,37.55000687,126.91482544,ST-7


In [17]:
# retrieving all data
startnum = [[1, 1000], [1001, 2000]]
df = pd.DataFrame()
for pair in startnum:
    url = f'http://openapi.seoul.go.kr:8088/{apikey}/json/bikeList/{pair[0]}/{pair[1]}/'
    json = requests.get(url).json()
    part = pd.DataFrame(json['rentBikeStatus']['row'])
    df = df.append(part)

df.shape

(1572, 7)

In [20]:
df.tail()

,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId
567,16,3538. 서울숲 IT캐슬,1,0,37.55089188,127.04476166,ST-1659
568,12,3539. 서원마을,7,0,37.56381989,127.13284302,ST-1686
569,22,3541. 커먼그라운드,3,0,37.54124832,127.06604004,ST-1690
570,19,3542. 래미안 구의파크 스위트,14,5,37.54504013,127.08995819,ST-1722
571,17,3543. 세종대학교(학술정보원),0,0,37.55137253,127.07444000,ST-1760


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1572 entries, 0 to 571
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   rackTotCnt         1572 non-null   object
 1   stationName        1572 non-null   object
 2   parkingBikeTotCnt  1572 non-null   object
 3   shared             1572 non-null   object
 4   stationLatitude    1572 non-null   object
 5   stationLongitude   1572 non-null   object
 6   stationId          1572 non-null   object
dtypes: object(7)
memory usage: 98.2+ KB


In [22]:
df.to_csv('bicycle.csv', index = False)

In [23]:
df = pd.read_csv('bicycle.csv')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1572 entries, 0 to 1571
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rackTotCnt         1572 non-null   int64  
 1   stationName        1572 non-null   object 
 2   parkingBikeTotCnt  1572 non-null   int64  
 3   shared             1572 non-null   int64  
 4   stationLatitude    1572 non-null   float64
 5   stationLongitude   1572 non-null   float64
 6   stationId          1572 non-null   object 
dtypes: float64(2), int64(3), object(2)
memory usage: 86.1+ KB


# Map of Stations of Bicycle Sharing System

In [25]:
df.head()

,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId
0,7,101. (구)합정동 주민센터,4,14,37.549561,126.905754,ST-3
1,22,102. 망원역 1번출구 앞,7,0,37.555649,126.910629,ST-4
2,16,103. 망원역 2번출구 앞,3,0,37.554951,126.910835,ST-5
3,15,104. 합정역 1번출구 앞,0,0,37.550629,126.914986,ST-6
4,7,105. 합정역 5번출구 앞,1,0,37.550007,126.914825,ST-7


In [26]:
import folium

In [29]:
seoul = [37.5536067, 126.9674308]
m = folium.Map(location = seoul, zoom_start = 11)
m

In [43]:
seoul = [37.5536067, 126.9674308]
m = folium.Map(location = seoul, zoom_start = 11)

for i in df.index:
    name = df.loc[i, 'stationName']
    bikes = df.loc[i, 'parkingBikeTotCnt']
    folium.Marker(df.loc[i, ['stationLatitude', 'stationLongitude']].tolist(),
                 tooltip = f'{name}: {bikes}').add_to(m)
    
m

This process adds individual markers according to the lat & long location of each station and assigns the station name as a tooltip, which can be viewed by hovering over the marker with pointer

In [46]:
from folium.plugins import MarkerCluster
seoul = [37.5536067, 126.9674308]
m = folium.Map(location = seoul, zoom_start = 11)

marker_cluster = MarkerCluster().add_to(m)

for i in df.index:
    name = df.loc[i, 'stationName']
    bikes = df.loc[i, 'parkingBikeTotCnt']
    total = df.loc[i, 'rackTotCnt']
    
    if bikes <= 2:
        color = 'red'
    elif bikes/total > 0.5:
        color = 'blue'
    else:
        color = 'green'
    
    icon = folium.Icon(color = color, icon = 'info-sign')
    
    folium.Marker(df.loc[i, ['stationLatitude', 'stationLongitude']].tolist(),
                 tooltip = f'{name}: {bikes}',
                 icon = icon).add_to(marker_cluster)

m

By assigning icon to the marker with different colors, we can show the bike status of each station
* RED for less than 3 bikes at station
* BLUE for more than 50% of racks are filled
* GREEN for the rest

Also, because there are too many stations and is difficult to see, we group the icons with nearby stations, which has a positive and a negative:
* positive side is that it is <b>easier to see the number of stations in a specific area</b>, and it is also <b>easier to zoom into the area of interest</b> (which can be done by simply clicking on the area)
* the negative is that <b>one cannot identify the status of station of interest until he zooms in close onto the station</b>

In [47]:
m.save('bicycle.html')